## Airport Case Study
Looking at the effects of vectorization on calculations relating to airports.  Start by loading in the dataset which represents various airports in the world, along with their lat and lon positions.

In [2]:
import numpy as np
import pandas as pd

coord = (pd.read_csv("../TLC-Labs/airports.dat", index_col=['AIRPORT'], 
                     usecols=['AIRPORT','LATITUDE','LONGITUDE'])
         .groupby(level=0)
         .first()
         .dropna()
         .sample(n=500, random_state=42)
         .sort_index())

coord.head()

,LATITUDE,LONGITUDE
AIRPORT,,
2CA,35.745556,-119.236389
8F3,33.623889,-101.240833
A08,56.598056,-134.242778
A15,70.718611,-154.388333
A27,64.440000,-144.936389


Reindex the entire DataFrame into pairs of airports, such that every airport is paired with every other airport. 

A MultiIndex can be used to handle the fact we now have two pieces of data to uniquely identify each row - the desination and source airport.

In [3]:
idx = pd.MultiIndex.from_product([coord.index, coord.index], 
                                             names=['origin','dest'])

pairs = pd.concat([coord.add_suffix('_SRC').reindex(idx, level='origin'),
                   coord.add_suffix('_DST').reindex(idx, level='dest')],
                  axis=1)

pairs.head()

LATITUDE_SRC  LONGITUDE_SRC  LATITUDE_DST  LONGITUDE_DST
origin dest                                                          
2CA    2CA      35.745556    -119.236389     35.745556    -119.236389
       8F3      35.745556    -119.236389     33.623889    -101.240833
       A08      35.745556    -119.236389     56.598056    -134.242778
       A15      35.745556    -119.236389     70.718611    -154.388333
       A27      35.745556    -119.236389     64.440000    -144.936389

Using the MultiIndex, get a list of tuples, where each tuple is a pair-wise combination of the airport source-destinations.

In [5]:
idx = idx[idx.get_level_values(0) <= idx.get_level_values(1)]

idx

MultiIndex([('2CA', '2CA'),
            ('2CA', '8F3'),
            ('2CA', 'A08'),
            ('2CA', 'A15'),
            ('2CA', 'A27'),
            ('2CA', 'A56'),
            ('2CA', 'A57'),
            ('2CA', 'A66'),
            ('2CA', 'A6K'),
            ('2CA', 'A71'),
            ...
            ('ZMD', 'ZMD'),
            ('ZMD', 'ZNC'),
            ('ZMD', 'ZSL'),
            ('ZMD', 'ZXO'),
            ('ZNC', 'ZNC'),
            ('ZNC', 'ZSL'),
            ('ZNC', 'ZXO'),
            ('ZSL', 'ZSL'),
            ('ZSL', 'ZXO'),
            ('ZXO', 'ZXO')],
           names=['origin', 'dest'], length=125250)

The following function is a non-vectorized implementation that we might have produced of the great circle distance function, which can be used to find the distance, following a great circle around the curviture of the earth, between two points based on their lat and long positions.

In [6]:
import math

def gcd_py(lat_src, lng_src, lat_dst, lng_dst):
    earth_radius_km = 6373
    degs_to_rads = math.pi/180.0
    precision = 8 #dp
    
    theta_1 = (90-lat_src) * degs_to_rads
    theta_2 = (90-lat_dst) * degs_to_rads
    
    omega_1 = lng_src * degs_to_rads
    omega_2 = lng_dst * degs_to_rads
    
    cos = (math.sin(theta_1) * math.sin(theta_2) * 
           math.cos(omega_1 - omega_2) + 
           math.cos(theta_1) * math.cos(theta_2))
    
    cos = round(cos, precision)
    arc = math.acos(cos)
    
    return arc * earth_radius_km

We can now use this function, to calculate for every pair of airports, the distance between each airport.

In [10]:
%%time
r = pairs.apply(lambda x: gcd_py(x['LATITUDE_SRC'], x['LONGITUDE_SRC'], 
                                 x['LATITUDE_DST'], x['LATITUDE_DST']), axis=1)

Wall time: 4.91 s


In [8]:
r.head()

origin  dest
2CA     2CA     11658.209986
        8F3     11805.567309
        A08      9742.476890
        A15      8152.546542
        A27      8873.080964
dtype: float64

Notice the time taken - approximately 5 seconds - this is the time to beat!  

Our first approach might be to apply vectorization techqniues to the function gcd_py. The following function does this by replacing basic Python math library implementation with those from numpy instead. 

For example, notice that math.sin() calls have been replaced with np.sin(), and rather than doing the degrees to radians calculation using Python primitive operations (multiplcation) we now use the numpy function deg2rad() instead.

In [11]:
def gcd_vec(lat_src, lng_src, lat_dst, lng_dst):
    earth_radius_km = 6373
    
    theta_1 = np.deg2rad(90 - lat_src)
    theta_2 = np.deg2rad(90 - lat_dst)
    
    omega_1 = np.deg2rad(lng_src)
    omega_2 = np.deg2rad(lng_dst)
    
    cos = (np.sin(theta_1) * np.sin(theta_2) * np.cos(omega_1 - omega_2) +
          np.cos(theta_1) * np.cos(theta_2))
    
    arc = np.arccos(cos)
    
    return arc * earth_radius_km

So, what does the time taken look like now?

In [12]:
%%time
r = pairs.apply(lambda x: gcd_vec(x['LATITUDE_SRC'], x['LONGITUDE_SRC'], 
                                  x['LATITUDE_DST'], x['LATITUDE_DST']), axis=1)

Wall time: 7.48 s


In [13]:
r.head()

origin  dest
2CA     2CA     11658.209995
        8F3     11805.567299
        A08      9742.476919
        A15      8152.546524
        A27      8873.080977
dtype: float64

Oh dear! This isn't any better - in fact, it's worse!  

The problem is that whilst the function gcd_vec supports vectorization, we're not actually using it in a vectorized way. Notice that we're using apply(), and taking every pair and calling the function. This is still iteration.

We should avoid iteration in this style, and should at a minimum prefer the use of itertuples... so, let's see how that does. First, let's try it with the original Python implementation.

In [14]:
%%time
pd.Series([gcd_py(*x) for x in pairs.itertuples(index=False)], 
                                            index=pairs.index)

Wall time: 712 ms


origin  dest
2CA     2CA        0.000000
        8F3     1660.364858
        A08     2578.473725
        A15     4404.379276
        A27     3624.913383
                   ...     
ZXO     ZLO     4763.710791
        ZMD     9345.085453
        ZNC     1622.875654
        ZSL     6754.874826
        ZXO        0.000000
Length: 250000, dtype: float64

Okay! That's more like it.  Even with a non-vectorized implementation the use of itertuples rather than apply has given us a significan performance boost.

We are now in the ms teritory... but we still should be able to do better. What happens if we use itertuples with the vectorized implementation we created?

In [15]:
%%time
pd.Series([gcd_vec(*x) for x in pairs.itertuples(index=False)], index=pairs.index)

<ipython-input-11-33ed6328afff>:13: RuntimeWarning: invalid value encountered in arccos
  arc = np.arccos(cos)


Wall time: 3.06 s


origin  dest
2CA     2CA        0.000095
        8F3     1660.364741
        A08     2578.473679
        A15     4404.379250
        A27     3624.913415
                   ...     
ZXO     ZLO     4763.710784
        ZMD     9345.085432
        ZNC     1622.875569
        ZSL     6754.874795
        ZXO        0.000134
Length: 250000, dtype: float64

What? Slower - again!

At this point you might be wondering what is going on. The reality is that we're still not actually making use of the vectorized nature of the gcd_vec function. Even with itertuples, whilst more performant than a for loop or apply(), we're still fundamantelly iterating and repetidly calling the function.

The gcd_vec() function has more "overhead" - it calls np.deg2rad() for example rather than doing the math inline like our python implementation did. So what? Well, it's doing this for ever single pair at the moment - because we're using it iteratively.


We need to start thinking vectorization. We've built a vectorized function - use it with a vector.

In [18]:
%%time
r = gcd_vec(pairs['LATITUDE_SRC'], pairs['LONGITUDE_SRC'], 
            pairs['LATITUDE_DST'], pairs['LONGITUDE_DST'])

Wall time: 38.2 ms


In [19]:
r

origin  dest
2CA     2CA        0.000095
        8F3     1660.364741
        A08     2578.473679
        A15     4404.379250
        A27     3624.913415
                   ...     
ZXO     ZLO     4763.710784
        ZMD     9345.085432
        ZNC     1622.875569
        ZSL     6754.874795
        ZXO        0.000134
Length: 250000, dtype: float64

That's more like it!

Notice our code is easier to read (both the vectorized gcd_vec() function itself by making use of numpy functions and the call of the function, through the removal of list comprehensions, iteration utitilites, or lambda expressions) and is more performant.

We're now down to 16ms - beating anything we've done before. 

The difference? Instead of iterating over every element, we now pass the whole list (vector) in as a parameter, and our gcd_vec was written in such a way (through the use of NumPy functions) to be able to apply operations like sin(), cos(), etc. across an entire vector rather than using iteration itself internally. 

No we have one call to the function (so the function call overheads are minimally invasive), but the gains from the vectorization blow everything else out the water.